In [ ]:
import numpy as np
import numpy.random as rnd
import theano
import theano.tensor as T
import lasagne
import time
import os

#### Все места, где нужно дописать код отмечены TODO.

## Считывание и подготовка данных.

In [2]:
# Считываем текстовые данные: все файлы должны лежать в одной папке data. 
# Проверьте, что у вас все хорошо с кодировками и текст нормально считывается.
data = ""

for fname in os.listdir("data"):
    with open("data/"+fname) as fin:
        text = fin.read().decode('cp1251')
        data += text

In [3]:
print data[:200]

---------------------------------------------


 Гражданский кодекс
 Российской Федерации
 Часть третья



26 ноября 2001 года



 №146-ФЗ 

  Принят Государственной Думой 
  1 ноября 


In [4]:
# Для дальнейшей работы нам нужно текст перевести в числовой формат.
chars = list(set(data))
VOCAB_SIZE = len(chars)

char_to_id = { ch:id for id,ch in enumerate(chars) }
id_to_char = { id:ch for id,ch in enumerate(chars) }
data_ids = [char_to_id[ch] for ch in data]

## Вспомогательные функции

In [46]:
# Необходимые константы
NUM_EPOCHS = 100
NUM_BATCHES = 1000
BATCH_SIZE = 100
SEQ_LEN = 20
LEARNING_RATE = 0.01
GRAD_CLIP = 100

В процессе обучения мы будем для каждого символа входной последовательности предсказывать следующий символ. Таким образом на вход сети мы будем подавать последовательности длины SEQ_LEN и получать на выходе последовательности той же длины, но со сдвигом на один символ.

In [20]:
def generate_random_batch(source):
    """Функция, которая генерирует batch из BATCH_SIZE случайных подстрок текста source. 
    Каждая подстрока должна иметь длину SEQ_LEN.
    
    source - массив целых чисел - номеров символов в тексте (пример - data_ids)
    
    Вернуть нужно кортеж (X,y), где
    X - матрица, в которой каждая строка - подстрока длины SEQ_LEN (подается на вход сети)
    y - матрица, в которой каждая строка - подстрока длины SEQ_LEN, (ожидается на выходе сети)
    Таким образом, каждая строка в y должна соответсвовать строке в X со сдвигом на один символ вправо.
    Например, если X[0]='hell', то y[0]='ello'
    
    Убедитесь, что вы генерируете X и y, которые правильно соответствуют друг другу.
    Также убедитесь, что ваша функция не вылезает за край текста (самое начало или конец текста).
    """
    
    # TODO
    
    return X_batch, y_batch

In [21]:
a,b = generate_random_batch(data_ids)
print ''.join(id_to_char[id] for id in a[0,:])
print ''.join(id_to_char[id] for id in b[0,:])

е, мер по обеспечени
, мер по обеспечению


В процессе тестирования мы будем предсказывать следующий символ по SEQ_LEN предыдущих. 
Генерировать очередной символ в тестовой посдедовательнсоти можно разными способами:
1. max_sample_fn: брать символ с максимальной вероятностью
2. proportional_sample_fn: генерировать символ пропорционально вероятности
3. alpha_sample_fn: генерировать символ пропорционально вероятности со следующей предобраоткой: 
    logprobs/alpha, где alpha - "жадность" из (0,1] - чем меньше, тем ближе генерация к выбору максимума
    после взятия экспоненты такие вероятности нужно перенормировать

Для устойчивости вычислений наша сеть будет выдавать не вероятности, а их логарифмы, поэтому не забывайте в нужных местах брать от них exp.

In [83]:
def max_sample_fn(logprobs):
    return np.argmax(logprobs) 

def proportional_sample_fn(logprobs):
    # TODO

def alpha_sample_fn(logprobs):
    # TODO

def generate_seed():
    """Функция выбирает случайное начало поседовательности из data, 
    которую мы потом можем продолжать с помощью нейросети.
    """
    start = np.random.randint(0,len(data)-SEQ_LEN)
    seed_phrase = data[start:start+SEQ_LEN]
    return seed_phrase

def generate_sample(logprobs_fn,sample_fn,seed_phrase,N=100):
    """Функция генерирует случайный текст при помощи нейросети и печатает его
    
    logprobs_fn - функция, которая по входной последовательности длины SEQ_LEN 
        предсказывает логарифмы вероятностей посдледующего символа (см. функцию train)
    sample_fn - функция, выбирающая следующий символ одним из способов, описанных выше
    seed_phrase - начальная фраза, с которой мы начинаем генерировать
    N - размер генерируемого текста
    
    """
    
    # TODO
    
    print(random_snippet)

In [41]:
# Технические вещи

# Вспомогательная функция для запаковки результата обучения 
def pack(err, network,inp, target,train_fn, logprobs_fn):
    return {'err':err, 
        'network':network,
        'inp':inp, 
        'target':target,
        'train_fn':train_fn, 
        'logprobs_fn':logprobs_fn
           } 

# numerically stable log-softmax with crossentropy
def logsoftmax(x):
    xdev = x-x.max(1,keepdims=True)
    lsm = xdev - T.log(T.sum(T.exp(xdev),axis=1,keepdims=True))
    return lsm
def lsmCE(x,y):
    return -T.clip(x,-20,0)[T.arange(y.shape[0]), y]

## Нейронная сеть

In [35]:
def build_network(input_var=None, emb_size = 40, n_hidden = 100):
    """Функция строит простейшую рекуррентную сеть, которая состоит из следующих слоев:
    
    1. Входной слой размера [BATCH_SIZE, SEQ_LEN]
    2. Embedding для перевода кодировки символов в нормальное представление: VOCAB_SIZE -> emb_size
    3. Рекуррентный слой c n_hidden элементов на скрытом слое
    4. Reshape выхода рекуррентного слоя в [-1, n_hidden]
    5. Обычный полносвязный слой n_hidden -> VOCAB_SIZE с logsoftmax в качестве нелинейности
    
    """
    
    # TODO
    
    return network

In [47]:
def train(data_ids, emb_size, n_hidden, show = False):
    """Функция обучает нейросеть по данным data_ids
    Следует обратить внимание на следующее:
    1. Сеть будем учить NUM_EPOCHS эпох, в каждой из которых будет NUM_BATCHES батчей
    2. Для того, чтобы следить за процессом обучения будем считать средний loss 
        на всех батчах в эпохе и сохранять его в массив err. Также будем генерировать 
        последовательности из случайных seeds после каждой эпохи, для этого нужна будет функция logprobs_fn,
        которая по входу х размера [1, SEQ_LEN] будет выдавать вектор логарифмов вероятностей 
        для последующего символа размера [1, VOCAB_SIZE]. 
        Например, если x='hell', то нас интересует каким будет символ после второго l. 
    3. Так как мы вместо softmax используем logsoftmax, то в качестве loss для сети нужно использовать lsmCE
    4. Перед тем, как делать шаг по градиенту, будем ограничивать градиент по норме
        с помощью функции lasagne.updates.total_norm_constraint с ограничением на норму GRAD_CLIP
    
    """
    err=np.zeros(NUM_EPOCHS)
        
    print("Building network ...")
    # TODO
    print("The network has {} params".format(lasagne.layers.count_params(network)))
    
    # Функции для loss, updates, train_fn и logprobs_fn
    # В качестве метода оптимизации рекомендуется использовать adam
    # TODO
    
    print("Training ...")
    for epoch in xrange(NUM_EPOCHS):
        start_time = time.time()
        for batch in xrange(NUM_BATCHES):
            # TODO
        
        if show:
            seed = generate_seed()
            print "Seed: ",seed
            print "Max sample:"
            generate_sample(logprobs_fn, max_sample_fn, seed)
            print "Proportional sample:"
            generate_sample(logprobs_fn, proportional_sample_fn, seed)
        print("Epoch {} \t loss = {:.4f} \t time = {:.2f}s".
                      format(epoch, err[epoch], time.time() - start_time))
             
    return pack(err, network, inp, target, train_fn, logprobs_fn)

Перед тем, как запускать обучение с большим числом итераций и длинными последовательностями, попробуйте запустить его на десяток итераций с последовательнсотямит по 5 символов и проверьте, что у вас генерируются какие-то вменяемые слоги. При этом достатояно использовать довольно маленькую сеть.

In [48]:
model = train(data_ids, 40, 200, show = False)

Building network ...
The network has 72982 params
Training ...
Epoch 0 	 loss = 1.6287 	 time = 9.22s
Epoch 1 	 loss = 1.3190 	 time = 9.20s
Epoch 2 	 loss = 1.2823 	 time = 9.20s
Epoch 3 	 loss = 1.2688 	 time = 9.18s
Epoch 4 	 loss = 1.2614 	 time = 9.20s
Epoch 5 	 loss = 1.2556 	 time = 9.19s
Epoch 6 	 loss = 1.2549 	 time = 9.20s
Epoch 7 	 loss = 1.2506 	 time = 9.26s
Epoch 8 	 loss = 1.2486 	 time = 9.26s
Epoch 9 	 loss = 1.2522 	 time = 9.26s
Epoch 10 	 loss = 1.2497 	 time = 9.26s
Epoch 11 	 loss = 1.2453 	 time = 9.22s
Epoch 12 	 loss = 1.2447 	 time = 9.23s
Epoch 13 	 loss = 1.2475 	 time = 9.26s
Epoch 14 	 loss = 1.2507 	 time = 9.23s
Epoch 15 	 loss = 1.2400 	 time = 9.25s
Epoch 16 	 loss = 1.2489 	 time = 9.23s
Epoch 17 	 loss = 1.2654 	 time = 9.27s
Epoch 18 	 loss = 1.2436 	 time = 9.26s
Epoch 19 	 loss = 1.2525 	 time = 9.27s
Epoch 20 	 loss = 1.2448 	 time = 9.26s
Epoch 21 	 loss = 1.2487 	 time = 9.26s
Epoch 22 	 loss = 1.2433 	 time = 9.27s
Epoch 23 	 loss = 1.2510 	 

## Посмотрим что из этого вышло

In [88]:
seed = u"Каждый человек должен"
alpha = 0.5
sampling_fun = alpha_sample_fn
result_length = 300

generate_sample(model['logprobs_fn'],sampling_fun,seed,result_length)

Каждый человек должен, в пределов на катериумо от принцелях, при правонарушениях, в случае имущества и таможенных органов по правонарушения право предусмотрения или производить положение принятия и провода транспортного органов при на на в товары соответствии с подрядования составления с установление которых законом или


In [89]:
seed = u"В случае неповиновения"
alpha = 0.5
sampling_fun = alpha_sample_fn
result_length = 300

generate_sample(model['logprobs_fn'],sampling_fun,seed,result_length)

В случае неповиновения в отношения договором при на правонарушения имущество и не принимается ее принять авторавательного на договора или иные действия с можетственности правонарушении законной установления админислицам, должностного судом срок, принятие должностного договор и таки содержащими недействия подлежащим в соо


## Дополнительные пункты

1. Обучение более сложной модели и контроль переобучения. Попробуйте подобрать хорошую модель RNN для данной задачи. Для этого  проанализируйте качество работы модели в зависимости от ее размеров, попробуйте использовать многослойную сеть. Также нужно проконтролировать переобучение моделей. Для этого можно выделить тестовый кусок из текста и смотреть на то, как меняется loss на нем в процессе обучения. Если на графиках видно переобучение, то стоит добавить dropout слои в модель (обычный dropout до, между и после рекуррентных слоев). При использовании дропаута на стадии предсказания для нового объекта нужно ставить флаг deterministic=True.
2. LSTM и GRU архитектуры. Вместо обычной RNN попробуйте LSTM и GRU архитектуры и сравните получающиеся результаты для моделей нескольких разных размеров. Также сравните модели на данных с разной SEQ_LEN. 
4. Визуализация. Попробуйте провизуализировать результаты. Например, можно смотреть на то, какие буквы модель хорошо предсказывает, а в каких сильно не уверена. Это покажет что именно выучила модель лучше всего. Также можно попробовать смотреть на активации разных скрытых нейронов при прочтении текста (как у Андрея Карпатого).
5. Более сложные данные. Попробуйте обучить модель на более структурированных данных, например коде. Используйте LSTM и GRU сети, они хорошо улавливают структуру в данных. Проанализируйте результаты: выделите нейроны, активации которых "отвечают" за структуру в данных. Этот пункт, пожалуй, стоит пробовать только если у вас есть нормальный GPU.
6. Продвинутый дропаут. Запрограммировать RNN с продвинутым дропаутом из [Gal, Ghahramani, 2016]. Сравнить с обычным вариантом дропаута по нерекуррентным связям.